# Basics of Machine learning in julia

In [1]:
versioninfo()

Julia Version 0.6.0
Commit 903644385b (2017-06-19 13:05 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Xeon(R) CPU E5-1620 v2 @ 3.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, ivybridge)


In [2]:
using MLDataUtils
using MLDatasets

INFO: Recompiling stale cache file /Users/macpro/.julia/lib/v0.6/MLDataUtils.ji for module MLDataUtils.


##  Crosvalidation in DataUtils

The function **`MLDataUtils.kfolds`** has been implemented with 3 different input types in mind

- **`MLDataUtils.kfolds(Integer, K)`** 
- **`MLDataUtils.kfolds(Array, K)`**.
- **`MLDataUtils.kfolds((Array,Array), K)`**.


#### `MLDataUtils.kfolds(Integer, K)` 

- The first argument is an `integer`, it is assumed to be the number of examples.
- The method will return a tple of Arrays. The first element in the tuple corresponds to `K` different arrays containing the indicies for training. The second element of the tuple corresponds to `K` differnt arrays containing the indicies for validation.

```julia
folds = MLDataUtils.kfolds(15, 5)

typeof(folds)
# Tuple{Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

folds[1]
#5-element Array{Array{Int64,1},1}:
# [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# [1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15]
# [1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15]   
# [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]   
 
folds[2]
#5-element Array{UnitRange{Int64},1}:
#1:3  
#4:6  
#7:9  
#10:12
#13:15
```

#### `MLDataUtils.kfolds(Array, K)` 


- The first argument is a subtype of `AbstractArray`, which is assumed to contain examples as columns.
- The method will return views containing each of the folds.

```julia
X, Y = MLDataUtils.load_iris()
folds = MLDataUtils.KFolds(X, 10)

typeof(folds)
# MLDataPattern.FoldsView{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}},Array{Float64,2},LearnBase.ObsDim.Last,Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

for f in folds
    println(size(f[1]), size(f[2]))
end

# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)
```

#### `MLDataUtils.kfolds( (Array, Array) K)` 


- The first argument is a tuple of two subtypes of AbstractArray.
- The method will return views containing each of the folds.

##### Summary

If `folds = MLDataUtils.KFolds((X,Y), 10)`  then `folds[n_fold][n_tr_va][n_X_Y]` will return the fold specified by `n_fold` containing train or validation according to `n_tr_va` and the data or labels according to `n_X_Y`.

Example with iris data


```julia
X, Y = MLDataUtils.load_iris()
folds = MLDataUtils.KFolds((X,Y), 10)

typeof(folds)
# MLDataPattern.FoldsView{Tuple{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}},Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true},SubArray{String,1,Array{String,1},Tuple{UnitRange{Int64}},true}}},Tuple{Array{Float64,2},Array{String,1}},Tuple{LearnBase.ObsDim.Last,LearnBase.ObsDim.Last},Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

for f in folds
    println(size(f[1]), size(f[2]))
end

# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)
# (4, 120)(4, 30)


# fold 1: train data
folds[1][1][1]
# 4×120 SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false}:
# 4.8  5.4  5.2  5.5  4.9  5.0  5.5  4.9  …  6.8  6.7  6.7  6.3  6.5  6.2  5.9
# 3.1  3.4  4.1  4.2  3.1  3.2  3.5  3.6     3.2  3.3  3.0  2.5  3.0  3.4  3.0
# 1.6  1.5  1.5  1.4  1.5  1.2  1.3  1.4     5.9  5.7  5.2  5.0  5.2  5.4  5.1
# 0.2  0.4  0.1  0.2  0.2  0.2  0.2  0.1     2.3  2.5  2.3  1.9  2.0  2.3  1.8


# fold 1: train labels
folds[1][1][2]
# 30-element SubArray{String,1,Array{String,1},Tuple{UnitRange{Int64}},true}:
# "setosa"
# "setosa"
# "setosa"
 
 
# fold 1: validation data
folds[1][2][1]
# 4×30 SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}:
# 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  5.1  4.8  5.0  5.0  5.2  5.2  4.7
# 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.3  3.4  3.0  3.4  3.5  3.4  3.2
# 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     1.7  1.9  1.6  1.6  1.5  1.4  1.6
# 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     0.5  0.2  0.2  0.4  0.2  0.2  0.2
 
 
# fold 1: validation labels
folds[1][2][2]
#30-element SubArray{String,1,Array{String,1},Tuple{UnitRange{Int64}},true}:
# "setosa"
# "setosa"
 
```


#### Example with `MLDataUtils.kfolds(Integer, K)` 


In [3]:
folds = MLDataUtils.kfolds(15, 5)
folds[1]

5-element Array{Array{Int64,1},1}:
 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
 [1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15]
 [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15]
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15]   
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]   

In [4]:
typeof(folds)

Tuple{Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

In [5]:
#size(folds) doesn't work since its tuple, must write size(folds,2)

In [6]:
folds[2]

5-element Array{UnitRange{Int64},1}:
 1:3  
 4:6  
 7:9  
 10:12
 13:15


#### Example with **`MLDataUtils.kfolds(Array, K)`** 

In [7]:
X, Y = MLDataUtils.load_iris();
folds = kfolds(X,5);

In [8]:
folds

5-element FoldsView(::Array{Float64,2}, ::Array{Array{Int64,1},1}, ::Array{UnitRange{Int64},1}, ObsDim.Last()) with element type Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}}:
 ([4.8 5.4 … 6.2 5.9; 3.1 3.4 … 3.4 3.0; 1.6 1.5 … 5.4 5.1; 0.2 0.4 … 2.3 1.8], [5.1 4.9 … 5.2 4.7; 3.5 3.0 … 3.4 3.2; 1.4 1.4 … 1.4 1.6; 0.2 0.2 … 0.2 0.2])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [4.8 5.4 … 6.6 5.2; 3.1 3.4 … 2.9 2.7; 1.6 1.5 … 4.6 3.9; 0.2 0.4 … 1.3 1.4])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [5.0 5.9 … 5.6 5.5; 2.0 3.0 … 3.0 2.5; 3.5 4.2 … 4.1 4.0; 1.0 1.5 … 1.3 1.3])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [5.5 6.1 … 7.7 6.0; 2.6 3.0 … 2.6 2.2; 4.4 4.6 … 6.9 5.0; 1.2 1.4 … 2.3 1.5])
 ([5.1 4.9 … 7.7 6.0; 3.5 3.0 … 2.6 2.2;

In [9]:
typeof(folds)

MLDataPattern.FoldsView{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}},Array{Float64,2},LearnBase.ObsDim.Last,Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

In [10]:
size(folds)

(5,)

In [11]:
folds[1][2]

4×30 SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}:
 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  5.1  4.8  5.0  5.0  5.2  5.2  4.7
 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.3  3.4  3.0  3.4  3.5  3.4  3.2
 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     1.7  1.9  1.6  1.6  1.5  1.4  1.6
 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     0.5  0.2  0.2  0.4  0.2  0.2  0.2

In [12]:
folds[1][2]

4×30 SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}:
 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  5.1  4.8  5.0  5.0  5.2  5.2  4.7
 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.3  3.4  3.0  3.4  3.5  3.4  3.2
 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     1.7  1.9  1.6  1.6  1.5  1.4  1.6
 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     0.5  0.2  0.2  0.4  0.2  0.2  0.2

In [13]:
for f in folds
    println(size(f[1]), size(f[2]))
end

(4, 120)(4, 30)
(4, 120)(4, 30)
(4, 120)(4, 30)
(4, 120)(4, 30)
(4, 120)(4, 30)


#### Example with **`MLDataUtils.kfolds((Array,Array), K)`** 

In [14]:
folds = kfolds((X, Y), 5);

In [15]:
typeof(folds)

MLDataPattern.FoldsView{Tuple{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}},Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true},SubArray{String,1,Array{String,1},Tuple{UnitRange{Int64}},true}}},Tuple{Array{Float64,2},Array{String,1}},Tuple{LearnBase.ObsDim.Last,LearnBase.ObsDim.Last},Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

In [16]:
size(folds)

(5,)

In [17]:
for f in folds
    println(size(f[1][1]), size(f[1][2]))
end

(4, 120)(120,)
(4, 120)(120,)
(4, 120)(120,)
(4, 120)(120,)
(4, 120)(120,)


In [18]:
# fold 1: train data
folds[1][1][1];

In [19]:
# fold 1: train labels
folds[1][1][2];

In [20]:
# fold 1: validation data
folds[1][2][1];

In [21]:
# fold 1: validation labels
folds[1][2][2];

# Example MNIST

In [22]:
train_data, train_labels = MNIST.traindata()
train_data = reshape(train_data,784,60000);

test_data, test_labels = MNIST.testdata()
test_data = reshape(test_data,784,10000);

#### Encoding class labels from 1 to n_classes
train_labels .= train_labels .+ 1 ;
test_labels .= test_labels .+ 1 ;

println(size(train_data))
println(size(test_data))

(784, 60000)
(784, 10000)


In [23]:
n_samples = size(train_data,2)
n_features = size(train_data,1)
n_folds = 10
n_classes = 10

10


The following examples use views to avoid copying the data K times (one per fold). This makes the CV process much more manageble when de data is big compared to the number of threads in the machine divided by the amount of available RAM.



In [24]:
eltype_data = eltype(train_data)

Float64

### Using `kfolds(n_samples, n_folds)`



In [25]:
path_to_perceptron ="../MulticlassPerceptron/source/"
push!(LOAD_PATH, path_to_perceptron)
using MulticlassPerceptron
percep = MulticlassPerceptron.MPerceptron(eltype_data, n_classes, n_features)

Perceptron{Float64}(n_classes=10, n_features=784)

In [26]:
# What the folds print is not very understandable
folds = MLDataUtils.kfolds(n_samples, n_folds)

(Array{Int64,1}[[6001, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], [

In [27]:
# WHy is this and array of arrays and folds[2] an array of unit ranges?
folds[1]

10-element Array{Array{Int64,1},1}:
 [6001, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000]                             
 [1, 2, 3, 4, 5, 6,

In [28]:
folds[2]

10-element Array{UnitRange{Int64},1}:
 1:6000     
 6001:12000 
 12001:18000
 18001:24000
 24001:30000
 30001:36000
 36001:42000
 42001:48000
 48001:54000
 54001:60000

In [29]:
y_signal_placeholder = zeros(n_classes)
fold_results = Array([])

for n_fold in 1:n_folds
    
    println("Fold: ",n_fold)
    X_tr_view = view(train_data, :, folds[1][n_fold])
    y_tr_view = view(train_labels,  folds[1][n_fold])
    X_va_view = view(train_data, :, folds[2][n_fold])
    y_va_view = view(train_labels,  folds[2][n_fold])
    
    percep = MulticlassPerceptron.MPerceptron(eltype_data, n_classes, n_features)
    
    MulticlassPerceptron.fit!(percep, X_tr_view, y_tr_view;
                              n_epochs=15, print_flag=true)
    
    # Evaluate
    y_tr_hat = Array([])
    @inbounds for m in 1:size(X_tr_view,2)
       push!(y_tr_hat, MulticlassPerceptron.predict(percep, view(X_tr_view, :, m)))   
    end
    acc_tr = accuracy(y_tr_hat, y_tr_view)
    println("train acc: ", acc_tr)

    y_va_hat = Array([])
    @inbounds for m in 1:size(X_va_view,2)
       push!(y_va_hat, MulticlassPerceptron.predict(percep, view(X_va_view, :, m)))   
    end
    acc_va = accuracy(y_va_hat, y_va_view)
    println("valid acc: ", acc_va)
    
    push!(fold_results, (acc_tr, acc_va))
    
end

Fold: 1
train acc: 0.8759074074074074
valid acc: 0.871
Fold: 2
train acc: 0.8769814814814815
valid acc: 0.8635
Fold: 3
train acc: 0.8844444444444445
valid acc: 0.8588333333333333
Fold: 4
train acc: 0.8857777777777778
valid acc: 0.8858333333333334
Fold: 5
train acc: 0.8840370370370371
valid acc: 0.8668333333333333
Fold: 6
train acc: 0.8887037037037037
valid acc: 0.866
Fold: 7
train acc: 0.8636296296296296
valid acc: 0.8493333333333334
Fold: 8
train acc: 0.8778518518518519
valid acc: 0.8588333333333333
Fold: 9
train acc: 0.888962962962963
valid acc: 0.8696666666666667
Fold: 10
train acc: 0.8827777777777778
valid acc: 0.8923333333333333


### Using `kfolds((X,Y), n_folds)`


In [30]:
y_signal_placeholder = zeros(n_classes)
fold_results = Array([])

0-element Array{Any,1}

In [31]:
y_signal_placeholder = zeros(n_classes)
fold_results = Array([])

n_fold = 0
for ((X_tr, y_tr), (X_va, y_va)) in kfolds((train_data, train_labels); k = 10)
    n_fold += 1 
    println("Fold: ", n_fold)
    
    percep = MulticlassPerceptron.MPerceptron(eltype_data, n_classes, n_features)
    MulticlassPerceptron.fit!(percep, X_tr, y_tr;
                              n_epochs=15, print_flag=true)
    
    # Evaluate
    y_tr_hat = Array([])
    @inbounds for m in 1:size(X_tr,2)
        push!(y_tr_hat, MulticlassPerceptron.predict(percep, X_tr[:, m]))   
    end
    acc_tr = accuracy(y_tr_hat, y_tr)
    println("train acc: ", acc_tr)

    y_va_hat = Array([])
    @inbounds for m in 1:size(X_va,2)
       push!(y_va_hat, MulticlassPerceptron.predict(percep, X_va[:, m]))   
    end
    acc_va = accuracy(y_va_hat, y_va)
    println("valid acc: ", acc_va)
    
    push!(fold_results, (acc_tr, acc_va))
    
end

Fold: 1Epoch: 2 	 Accuracy: 0.878
train acc: 0.8915555555555555
valid acc: 0.8873333333333333
Fold: 2
train acc: 0.8769814814814815
valid acc: 0.8635
Fold: 3
train acc: 0.8844444444444445
valid acc: 0.8588333333333333
Fold: 4
train acc: 0.8857777777777778
valid acc: 0.8858333333333334
Fold: 5
train acc: 0.8840370370370371
valid acc: 0.8668333333333333
Fold: 6
train acc: 0.8887037037037037
valid acc: 0.866
Fold: 7
train acc: 0.8636296296296296
valid acc: 0.8493333333333334
Fold: 8
train acc: 0.8778518518518519
valid acc: 0.8588333333333333
Fold: 9
train acc: 0.888962962962963
valid acc: 0.8696666666666667
Fold: 10
train acc: 0.8827777777777778
valid acc: 0.8923333333333333


# Crossvalidation using map

In [32]:
X_tr_folds = kfolds(train_data ; k = 10);
y_tr_folds = kfolds(train_labels ; k = 10);

In [33]:
size(X_tr_folds[1][1]),size(X_tr_folds[1][2])

((784, 54000), (784, 6000))

In [34]:
size(y_tr_folds[1][1]),size(y_tr_folds[1][2])

((54000,), (6000,))

In [35]:
function train_evaluate(model, X_fold, y_fold)
    
    MulticlassPerceptron.fit!(model, X_fold[1], y_fold[1];
                              n_epochs=15, print_flag=true)
    
    # Evaluate
    y_tr_hat = Array([])
    @inbounds for m in 1:size(X_fold[1],2)
       push!(y_tr_hat, MulticlassPerceptron.predict(model, X_fold[1][:, m]))   
    end
    acc_tr = accuracy(y_tr_hat, y_fold[1])
    println("train acc: ", acc_tr)

    y_va_hat = Array([])
    @inbounds for m in 1:size(X_fold[2],2)
       push!(y_va_hat, MulticlassPerceptron.predict(model,  X_fold[2][:, m]))   
    end
    acc_va = accuracy(y_va_hat, y_fold[2])
    println("valid acc: ", acc_va)
    
    return (acc_tr, acc_va) 
end

train_evaluate (generic function with 1 method)

In [36]:
# We can create K copies of the model, one per fold, and train all of them 
models = [percep for i in 1:10]

10-element Array{MulticlassPerceptron.MPerceptron{Float64},1}:
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)
 Perceptron{Float64}(n_classes=10, n_features=784)

In [37]:
@time cvresults = map(train_evaluate, models, X_tr_folds, y_tr_folds)

train acc: 0.8850555555555556
valid acc: 0.881
train acc: 0.8817222222222222
valid acc: 0.868
train acc: 0.9086296296296297
valid acc: 0.8811666666666667
train acc: 0.8794259259259259
valid acc: 0.8791666666666667
train acc: 0.8805555555555555
valid acc: 0.8678333333333333
train acc: 0.889racy: 0.904
valid acc: 0.8658333333333333
train acc: 0.8836666666666667
valid acc: 0.8706666666666667
train acc: 0.8920370370370371
valid acc: 0.8731666666666666
train acc: 0.8765555555555555
valid acc: 0.8606666666666667
train acc: 0.867574074074074
valid acc: 0.8823333333333333
 56.514634 seconds (105.90 M allocations: 16.093 GiB, 4.21% gc time)


10-element Array{Tuple{Float64,Float64},1}:
 (0.885056, 0.881)   
 (0.881722, 0.868)   
 (0.90863, 0.881167) 
 (0.879426, 0.879167)
 (0.880556, 0.867833)
 (0.889, 0.865833)   
 (0.883667, 0.870667)
 (0.892037, 0.873167)
 (0.876556, 0.860667)
 (0.867574, 0.882333)

# Crossvalidation using pmap

In [38]:
nprocs()

1

In [39]:
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [40]:
nprocs()

9

In [41]:
@everywhere function train_evaluate(model, X_fold, y_fold)
    
    MulticlassPerceptron.fit!(model, X_fold[1], y_fold[1];
                              n_epochs=15, print_flag=true)
    
    # Evaluate
    y_tr_hat = Array([])
    @inbounds for m in 1:size(X_fold[1],2)
       push!(y_tr_hat, MulticlassPerceptron.predict(model, X_fold[1][:, m]))   
    end
    acc_tr = accuracy(y_tr_hat, y_fold[1])
    println("train acc: ", acc_tr)

    y_va_hat = Array([])
    @inbounds for m in 1:size(X_fold[2],2)
       push!(y_va_hat, MulticlassPerceptron.predict(model,  X_fold[2][:, m]))   
    end
    acc_va = accuracy(y_va_hat, y_fold[2])
    println("valid acc: ", acc_va)
    
    return (acc_tr, acc_va) 
end

@everywhere using MulticlassPerceptron

In [42]:
@time cvresults = pmap(train_evaluate, models, X_tr_folds, y_tr_folds)

train acc: 0.904962962962963ccuracy: 0.901
	From worker 2:	valid acc: 0.8991666666666667
train acc: 0.8894814814814814curacy: 0.9
train acc: 0.8736666666666667curacy: 0.903
train acc: 0.8825acy: 0.902Accuracy: 0.902
train acc: 0.8837592592592592curacy: 0.904
	From worker 3:	valid acc: 0.8785
	From worker 8:	valid acc: 0.8601666666666666
	From worker 5:	valid acc: 0.8791666666666667
train acc: 0.8808888888888889curacy: 0.902
	From worker 9:	valid acc: 0.8635
	From worker 4:	valid acc: 0.8648333333333333
train acc: 0.8897592592592592curacy: 0.901
	From worker 6:	valid acc: 0.88
train acc: 0.8932222222222223curacy: 0.902
	From worker 7:	valid acc: 0.8701666666666666
train acc: 0.907962962962963ccuracy: 0.902
	From worker 2:	valid acc: 0.8856666666666667
train acc: 0.8780185185185185curacy: 0.901
	From worker 3:	valid acc: 0.8853333333333333
 25.583078 seconds (1.18 M allocations: 3.583 GiB, 3.66% gc time)


10-element Array{Tuple{Float64,Float64},1}:
 (0.904963, 0.899167)
 (0.889481, 0.8785)  
 (0.883759, 0.864833)
 (0.8825, 0.879167)  
 (0.889759, 0.88)    
 (0.893222, 0.870167)
 (0.873667, 0.860167)
 (0.880889, 0.8635)  
 (0.907963, 0.885667)
 (0.878019, 0.885333)

# Saving model to disc

In [43]:
using JLD

# Comments/Stuff to consider

In [44]:
fieldnames(folds)

2-element Array{Int64,1}:
 1
 2

In [45]:
folds.data

LoadError: [91mtype Tuple has no field data[39m

#### Shouldn't a FoldsView be an array of Fold types? Now it's a type containing tuples

- Have a fold type  with `train_fold/validation_fold` field to make the code more readable.
```
X_fold[1] -> X_vold.tr_fold
X_fold[2] -> X_vold.va_fold
```



#### Make Prints of a FoldsView more understandable 


Example with iris data does not seem very understandable

```julia
X_iris, Y_iris = MLDataUtils.load_iris()
folds = kfolds(X_iris)
```
```
5-element FoldsView(::Array{Float64,2}, ::Array{Array{Int64,1},1}, ::Array{UnitRange{Int64},1}, ObsDim.Last()) with element type Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}}:
 ([4.8 5.4 … 6.2 5.9; 3.1 3.4 … 3.4 3.0; 1.6 1.5 … 5.4 5.1; 0.2 0.4 … 2.3 1.8], [5.1 4.9 … 5.2  4.7; 3.5 3.0 … 3.4 3.2; 1.4 1.4 … 1.4 1.6; 0.2 0.2 … 0.2 0.2])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [4.8 5.4 … 6.6  5.2; 3.1 3.4 … 2.9 2.7; 1.6 1.5 … 4.6 3.9; 0.2 0.4 … 1.3 1.4])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [5.0 5.9 … 5.6 5.5; 2.0 3.0 … 3.0 2.5; 3.5 4.2 … 4.1 4.0; 1.0 1.5 … 1.3 1.3])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [5.5 6.1 … 7.7  6.0; 2.6 3.0 … 2.6 2.2; 4.4 4.6 … 6.9 5.0; 1.2 1.4 … 2.3 1.5])
 ([5.1 4.9 … 7.7 6.0; 3.5 3.0 … 2.6 2.2; 1.4 1.4 … 6.9 5.0; 0.2 0.2 … 2.3 1.5], [6.9 5.6 … 6.2 5.9; 3.2 2.8 … 3.4 3.0; 5.7 4.9 … 5.4 5.1; 2.3 2.0 … 2.3 1.8])
```

Maybe if it printed something like...

```
MLDataPattern.FoldsView{data=X_iris, n_samples=150, n_folds=10, tr_sizes = (4,135), va_sizes = (4,15)}
```


When we ask for a type we get
```julia
typeof(folds)
```
``` 
MLDataPattern.FoldsView{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}},Array{Float64,2},LearnBase.ObsDim.Last,Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}
```



would give us an overview of the crosvalidation procedure with interesting information

- There are `n_samples`(we can detect easily if we gave a transposed matrix), 
- There are `n_folds` 
- Sizes of the train and validation folds





In [46]:
X_iris, Y_iris = MLDataUtils.load_iris()

([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], String["setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa", "setosa"  …  "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica", "virginica"], String["Sepal length", "Sepal width", "Petal length", "Petal width"])

In [47]:
iris_folds = kfolds(X_iris,10)

10-element FoldsView(::Array{Float64,2}, ::Array{Array{Int64,1},1}, ::Array{UnitRange{Int64},1}, ObsDim.Last()) with element type Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}}:
 ([5.7 5.4 … 6.2 5.9; 4.4 3.9 … 3.4 3.0; 1.5 1.3 … 5.4 5.1; 0.4 0.4 … 2.3 1.8], [5.1 4.9 … 4.3 5.8; 3.5 3.0 … 3.0 4.0; 1.4 1.4 … 1.1 1.2; 0.2 0.2 … 0.1 0.2])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [5.7 5.4 … 5.2 4.7; 4.4 3.9 … 3.4 3.2; 1.5 1.3 … 1.4 1.6; 0.4 0.4 … 0.2 0.2])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [4.8 5.4 … 5.0 5.1; 3.1 3.4 … 3.5 3.8; 1.6 1.5 … 1.6 1.9; 0.2 0.4 … 0.6 0.4])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], [4.8 5.1 … 6.6 5.2; 3.0 3.8 … 2.9 2.7; 1.4 1.6 … 4.6 3.9; 0.3 0.2 … 1.3 1.4])
 ([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0

In [48]:
iris_folds.data

4×150 Array{Float64,2}:
 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  6.8  6.7  6.7  6.3  6.5  6.2  5.9
 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.2  3.3  3.0  2.5  3.0  3.4  3.0
 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     5.9  5.7  5.2  5.0  5.2  5.4  5.1
 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     2.3  2.5  2.3  1.9  2.0  2.3  1.8

In [49]:
size(folds.data,2)

LoadError: [91mtype Tuple has no field data[39m

In [50]:
size(folds.train_indices)

LoadError: [91mtype Tuple has no field train_indices[39m

In [51]:
size(folds.train_indices[1])

LoadError: [91mtype Tuple has no field train_indices[39m

In [52]:
size(folds[1][1]), size(folds[1][2])

((54000,), (54000,))

In [53]:
using Base
function Base.show(io::IO, fv::FoldsView)
    name_array = fv.dataArray
    n_samples  = size(fv.data, 2)
    n_folds = size(fv.train_indices)
    tr_sizes = size(folds[1])
    va_sizes = size(folds[2])
    print(io, "FoldsView(data=name_array ,n_samples=$n_samples, n_folds=$n_folds, tr_sizes=$tr_sizes, va_sizes=$va_sizes ) ")
end


In [54]:
f  = kfolds(train_data ; k = 10);

In [55]:
f.data

784×60000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [56]:
f

10-element FoldsView(::Array{Float64,2}, ::Array{Array{Int64,1},1}, ::Array{UnitRange{Int64},1}, ObsDim.Last()) with element type Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}}:
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])
 ([0.0 

In [57]:
X, Y = MLDataUtils.load_iris()
folds = MLDataUtils.kfolds(X, 10)

typeof(folds)
# MLDataPattern.FoldsView{Tuple{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}},Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true},SubArray{String,1,Array{String,1},Tuple{UnitRange{Int64}},true}}},Tuple{Array{Float64,2},Array{String,1}},Tuple{LearnBase.ObsDim.Last,LearnBase.ObsDim.Last},Array{Array{Int64,1},1},Array{UnitRange{Int64},1}


MLDataPattern.FoldsView{Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}},Array{Float64,2},LearnBase.ObsDim.Last,Array{Array{Int64,1},1},Array{UnitRange{Int64},1}}

In [58]:
typeof(folds[1])

Tuple{SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}}

#### How could we extend the KFold type  ...

Maybe a KFold type could contain fieldnames for ``train_results`` and ``validation_results`` as well as the metric used.

``train_results`` and ``validation_results`` could start by defualt with empty arrays which would be occupied once the CV is done.

Another way could be to add a type to keep that info organized. Such a ``FoldsResults``
 type.

In [59]:
type FoldsResults
    metric::Function
    train_results::Array
    validation_results::Array
end

In [60]:
fold_res = FoldsResults(accuracy, [x[1] for x in fold_results],  [x[2] for x in fold_results])

FoldsResults(MulticlassPerceptron.accuracy, [0.891556, 0.876981, 0.884444, 0.885778, 0.884037, 0.888704, 0.86363, 0.877852, 0.888963, 0.882778], [0.887333, 0.8635, 0.858833, 0.885833, 0.866833, 0.866, 0.849333, 0.858833, 0.869667, 0.892333])

In [61]:
fold_res.train_results

10-element Array{Float64,1}:
 0.891556
 0.876981
 0.884444
 0.885778
 0.884037
 0.888704
 0.86363 
 0.877852
 0.888963
 0.882778

In [62]:
fold_res.validation_results

10-element Array{Float64,1}:
 0.887333
 0.8635  
 0.858833
 0.885833
 0.866833
 0.866   
 0.849333
 0.858833
 0.869667
 0.892333